In [13]:
!pip install langchain
!pip install langchain-openai
!pip install langgraph
!pip install grandalf

In [7]:
import json

api_key = None
with open('config.json') as f:
    data = json.load(f)
    api_key = data['openai']['api_key']

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


#llm = ChatOpenAI()
llm = ChatOpenAI(api_key=api_key)
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [16]:
from typing import List, Sequence

from dotenv import load_dotenv
load_dotenv()

from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

REFLECT = "reflect"
GENERATE = "generate"


def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]


builder = MessageGraph()
builder.add_node(GENERATE, generation_node)
builder.add_node(REFLECT, reflection_node)
builder.set_entry_point(GENERATE)


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return REFLECT


builder.add_conditional_edges(GENERATE, should_continue)
builder.add_edge(REFLECT, GENERATE)

graph = builder.compile()
print(graph.get_graph().draw_mermaid())
graph.get_graph().print_ascii()

if __name__ == "__main__":
    print("Hello LangGraph")
    inputs = HumanMessage(content="""Make this tweet better:"
                                    @LangChainAI
            — newly Tool Calling feature is seriously underrated.

            After a long wait, it's  here- making the implementation of agents across different models with function calling - super easy.

            Made a video covering their newest blog post

                                  """)
    
    
    response = graph.invoke(inputs)

    print(response)

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

          +-----------+            
          | __start__ |            
          +-----------+            
                *                  
                *                  
                *                  
          +----------+             
          | generate |             
          +----------+             
          ***        ...           
         *              .          
       **                ..        
+---------+           +---------+  
| reflect |           | __end__ |  
+---------+           +---------+  
Hello LangGraph
[HumanMessage(content='Make this tweet better:"\n                        

In [19]:
# Function to convert the conversation to Markdown format
def convert_to_markdown(conversation):
    md_output = "# **Tweet Improvement Discussion**\n\n"
    
    for i, msg in enumerate(conversation):
        role = "**Human:**" if isinstance(msg, HumanMessage) else "**AI:**"
        step_number = f"## **{i + 1}️⃣ {'Human' if isinstance(msg, HumanMessage) else 'AI'} Message**\n"
        formatted_msg = f"```md\n{msg.content}\n```\n"

        md_output += f"{step_number}{role}\n{formatted_msg}\n"
    
    return md_output

# Convert conversation to Markdown
markdown_text = convert_to_markdown(response)  # Ensure `response` is a list of HumanMessage/AIMessage objects

# Save to a Markdown file
with open("conversation.md", "w", encoding="utf-8") as f:
    f.write(markdown_text)

print("✅ Markdown file 'conversation.md' has been generated!")


✅ Markdown file 'conversation.md' has been generated!
